# Relatório de Estatística Descritiva: Titanic (D1) e Eleições (D2)

**Aluno:** Carlos Emanuel de Sousa Silva  
**Matrícula:** 2528049  
**Data:** 27/12/2025  
**Disciplina:** Estatística Descritiva  
**Professor:** Hygor Piaget M. Melo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração visual para os gráficos
sns.set(style="whitegrid")
%matplotlib inline 

# Leitura dos datasets
df_titanic = pd.read_csv("titanic.csv")
df_eleicoes = pd.read_csv("eleicoes.csv", sep=",")

# Padronização do dataset de Eleições
df_eleicoes.columns = ["UF", "Candidate_Number", "Money", "Votes"]
df_eleicoes["UF"] = df_eleicoes["UF"].str.strip()
df_eleicoes["Money"] = df_eleicoes["Money"].astype(float)
df_eleicoes["Votes"] = df_eleicoes["Votes"].astype(int)

##  Análise do Dataset: Titanic (D1)

Nesta seção, analisamos o perfil dos passageiros. As variáveis são:
* **Qualitativas:** Sex (Sexo), Survived (Sobrevivência), Pclass (Classe).
* **Quantitativas:** Age (Idade) e Fare (Tarifa).

In [ ]:
colunas_qualitativas = ['Sex', 'Pclass', 'Survived']

for col in colunas_qualitativas:
    plt.figure(figsize=(7, 4))
    sns.countplot(x=col, data=df_titanic, palette='viridis')
    plt.title(f"Distribuição da Variável: {col}")
    plt.ylabel("Frequência Absoluta")
    plt.show()

Os **gráficos de barras** mostram que a **moda** da amostra é composta por passageiros do sexo masculino, da 3ª classe e de não sobreviventes. Isso ajuda a descrever a composição demográfica predominante no desastre.

In [ ]:
# Tendência Central e Dispersão
estatisticas_quant = df_titanic[['Age', 'Fare']].agg(['mean', 'median', 'std', 'min', 'max'])
print("--- MEDIDAS DE TENDÊNCIA CENTRAL E DISPERSÃO ---")
display(estatisticas_quant)

# Medidas Separatrizes para Age
percentis_age = df_titanic['Age'].quantile([0.10, 0.25, 0.50, 0.75, 0.90])
print("\n--- MEDIDAS SEPARATRIZES (AGE) ---")
display(percentis_age.to_frame(name="Valor"))

A variável Fare apresenta um **desvio padrão** muito superior à sua média, indicando alta **dispersão** e desigualdade nos preços. Já a variável Age apresenta média e mediana próximas, sugerindo baixa **assimetria**.
As **medidas separatrizes** mostram que 10% dos passageiros (Percentil 10) tinham menos de 14 anos.

In [ ]:
# Histograma
plt.figure(figsize=(8, 5))
sns.histplot(df_titanic['Age'], bins=20, kde=True, color='skyblue')
plt.title("Histograma de Frequência - Idade (Age)")
plt.show()

# Dispersão
plt.figure(figsize=(8, 5))
sns.scatterplot(x='Age', y='Fare', data=df_titanic, alpha=0.5)
corr_age_fare = df_titanic['Age'].corr(df_titanic['Fare'])
plt.title(f"Dispersão: Idade vs Tarifa\nCorrelação de Pearson = {corr_age_fare:.4f}")
plt.show()

## Análise do Dataset: Eleições (D2)

Diferente do Titanic, este dataset apresenta dados com extrema amplitude, exigindo o uso de **escala logarítmica** para visualização adequada das distribuições e correlações.

In [ ]:
estados = ["SP", "RJ", "CE"]

for uf in estados:
    dados_votos = df_eleicoes[(df_eleicoes["UF"] == uf) & (df_eleicoes["Votes"] > 0)]
    
    plt.figure(figsize=(8, 4))
    plt.hist(dados_votos["Votes"], bins=30)
    plt.xscale("log")
    plt.yscale("log")
    plt.title(f"Histograma de Votos (Log-Log) - {uf}")
    plt.xlabel("Votos (log)")
    plt.ylabel("Frequência (log)")
    plt.show()

O uso da **escala logarítmica** revela uma forte **assimetria positiva**. Há uma grande **concentração** de candidatos com poucos votos e uma "cauda longa" que representa candidatos com votações expressivas (**valores extremos/outliers**).

In [ ]:
for uf in estados:
    dados = df_eleicoes[(df_eleicoes["UF"] == uf) & (df_eleicoes["Money"] > 0) & (df_eleicoes["Votes"] > 0)]
    corr = dados["Money"].corr(dados["Votes"])
    
    plt.figure(figsize=(8, 5))
    sns.regplot(x="Money", y="Votes", data=dados, logx=True, line_kws={'color': 'red'})
    plt.xscale("log")
    plt.yscale("log")
    plt.title(f"Dispersão Dinheiro x Votos - {uf}\nCorrelação de Pearson = {corr:.2f}")
    plt.show()

Observa-se uma **correlação de Pearson** positiva moderada a forte. O padrão visual no gráfico de dispersão indica que, em escala logarítmica, o aumento de investimento financeiro está linearmente associado ao aumento de votos.

In [ ]:
# Correlação por Estado
correlacao_uf = df_eleicoes[(df_eleicoes["Money"] > 0) & (df_eleicoes["Votes"] > 0)].groupby("UF", group_keys=False).apply(lambda x: x["Money"].corr(x["Votes"]))
print("--- CORRELAÇÃO POR ESTADO ---")
display(correlacao_uf.to_frame(name="Correlação"))

# Medidas de Tendência Central e Dispersão por UF
resumo_estados = df_eleicoes.groupby("UF").agg({
    "Votes": ["mean", "median", "std", "min", "max", lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
    "Money": ["mean", "median", "std", "min", "max", lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)]
})
print("\n--- RESUMO DE MEDIDAS POR ESTADO ---")
display(resumo_estados)